In [6]:
from transformers.utils import logging
from pathlib import Path

cache_dir = Path.home() / ".cache/huggingface/transformers"
print("Hugging Face model cache directory:", cache_dir)


Hugging Face model cache directory: /Users/nuzkhan/.cache/huggingface/transformers


In [ ]:
#! pip install tiktoken protobuf
#! pip install torch transformers


You should consider upgrading via the '/Users/nuzkhan/Downloads/medical chatbot/medchatbot/bin/python3 -m pip install --upgrade pip' command.


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from pathlib import Path

model_path = Path.home() / "mistral_models" / "7B-Instruct-v0.3"

# Use trust_remote_code=True to let Transformers load Mistral custom tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

# Test generation
prompt = "Explain COVID-19 in 2 sentences."
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=100)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)


ValueError: Converting from SentencePiece and Tiktoken failed, if a converter for SentencePiece is available, provide a model path with a SentencePiece tokenizer.model file.Currently available slow->fast converters: ['AlbertTokenizer', 'BartTokenizer', 'BarthezTokenizer', 'BertTokenizer', 'BigBirdTokenizer', 'BlenderbotTokenizer', 'CamembertTokenizer', 'CLIPTokenizer', 'CodeGenTokenizer', 'ConvBertTokenizer', 'DebertaTokenizer', 'DebertaV2Tokenizer', 'DistilBertTokenizer', 'DPRReaderTokenizer', 'DPRQuestionEncoderTokenizer', 'DPRContextEncoderTokenizer', 'ElectraTokenizer', 'FNetTokenizer', 'FunnelTokenizer', 'GPT2Tokenizer', 'HerbertTokenizer', 'LayoutLMTokenizer', 'LayoutLMv2Tokenizer', 'LayoutLMv3Tokenizer', 'LayoutXLMTokenizer', 'LongformerTokenizer', 'LEDTokenizer', 'LxmertTokenizer', 'MarkupLMTokenizer', 'MBartTokenizer', 'MBart50Tokenizer', 'MPNetTokenizer', 'MobileBertTokenizer', 'MvpTokenizer', 'NllbTokenizer', 'OpenAIGPTTokenizer', 'PegasusTokenizer', 'Qwen2Tokenizer', 'Realm

In [7]:
from huggingface_hub import snapshot_download
from pathlib import Path

# Path where Gemma will be saved
gemma_path = Path.home() / "gemma_models" / "gemma-3-1b-it"
gemma_path.mkdir(parents=True, exist_ok=True)

# Download the Gemma 3 instruction-tuned 1B model
snapshot_download(
    repo_id="google/gemma-3-1b-it",  # Gemma 3 1B instruction-tuned
    local_dir=gemma_path,
    allow_patterns=["*"]  # download all files
)

print(f"✅ Gemma 3 model downloaded to: {gemma_path}")


Python(76293) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Fetching 10 files: 100%|██████████| 10/10 [00:01<00:00,  5.80it/s]

✅ Gemma 3 model downloaded to: /Users/nuzkhan/gemma_models/gemma-3-1b-it


In [8]:
from app.components.llm import load_llm
import torch

tokenizer, model = load_llm()
if tokenizer is None or model is None:
    print("❌ Failed to load LLM")
else:
    print("✅ LLM loaded successfully")

# Quick test generation
prompt = "Explain what a fever is in 2 sentences."
inputs = tokenizer(prompt, return_tensors="pt")
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=50, temperature=0.3)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated text:", text)


`torch_dtype` is deprecated! Use `dtype` instead!


✅ LLM loaded successfully
Generated text: Explain what a fever is in 2 sentences.

A fever is a temporary increase in body temperature caused by an infection or other illness. It's a sign that your body is fighting off harmful substances.

---

**Here are some additional points to consider:**

*   **Not always


In [10]:
# retriever.py
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

from app.components.llm import load_llm
from app.components.vector_store import load_vector_store, save_vector_store
from app.components.pdf_loader import load_pdf_files, create_text_chunks
from app.components.embeddings import get_embedding_model
from app.config.config import DB_FAISS_PATH
from app.common.logger import get_logger
from app.common.custom_exception import CustomException

logger = get_logger(__name__)

CUSTOM_PROMPT_TEMPLATE = """ Answer the following medical question in 2-3 lines maximum using only the information provided in the context.

Context:
{context}

Question:
{question}

Answer:
"""

def set_custom_prompt():
    return PromptTemplate(
        template=CUSTOM_PROMPT_TEMPLATE,
        input_variables=["context", "question"]
    )


def create_qa_chain():
    try:
        logger.info("Loading vector store for context...")
        db = load_vector_store()

        # If vector store is missing, create it automatically
        if db is None:
            logger.warning("Vector store not found. Creating a new one...")
            documents = load_pdf_files()
            if not documents:
                raise CustomException("No documents found to create vector store")
            text_chunks = create_text_chunks(documents)
            db = save_vector_store(text_chunks)
            if db is None:
                raise CustomException("Failed to create vector store")

        # Load Gemma 3 LLM
        tokenizer, llm_model = load_llm()
        if not tokenizer or not llm_model:
            raise CustomException("LLM not loaded")

        # Define a simple wrapper function for Gemma 3 to integrate with LangChain
        class GemmaWrapper:
            def __init__(self, tokenizer, model):
                self.tokenizer = tokenizer
                self.model = model

            def __call__(self, prompt: str):
                inputs = self.tokenizer(prompt, return_tensors="pt")
                with torch.no_grad():
                    outputs = self.model.generate(**inputs, max_new_tokens=256)
                return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        llm = GemmaWrapper(tokenizer, llm_model)

        # Create the RetrievalQA chain
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=db.as_retriever(search_kwargs={'k': 1}),
            return_source_documents=False,
            chain_type_kwargs={'prompt': set_custom_prompt()}
        )

        logger.info("✅ QA chain created successfully")
        return qa_chain

    except Exception as e:
        error_message = CustomException("Failed to create QA chain", e)
        logger.error(str(error_message))
        return None



In [11]:
import torch
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.schema import BaseRetriever

from app.components.llm import load_llm
from app.components.vector_store import load_vector_store, save_vector_store
from app.components.pdf_loader import load_pdf_files, create_text_chunks
from app.common.logger import get_logger
from app.common.custom_exception import CustomException

logger = get_logger(__name__)

CUSTOM_PROMPT_TEMPLATE = """ Answer the following medical question in 2-3 lines maximum using only the information provided in the context.

Context:
{context}

Question:
{question}

Answer:
"""

def set_custom_prompt():
    return PromptTemplate(
        template=CUSTOM_PROMPT_TEMPLATE,
        input_variables=["context", "question"]
    )

class GemmaLLMWrapper:
    """Wrap Gemma 3 for LangChain."""
    def __init__(self, tokenizer, model):
        self.tokenizer = tokenizer
        self.model = model

    def __call__(self, prompt: str, **kwargs):
        inputs = self.tokenizer(prompt, return_tensors="pt")
        with torch.no_grad():
            outputs = self.model.generate(**inputs, max_new_tokens=kwargs.get("max_new_tokens", 256))
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

def create_qa_chain():
    try:
        logger.info("Loading vector store for context...")
        db = load_vector_store()

        # Auto-create vectorstore if missing
        if db is None:
            logger.warning("Vector store not found. Creating a new one...")
            documents = load_pdf_files()
            if not documents:
                raise CustomException("No documents found to create vector store")
            text_chunks = create_text_chunks(documents)
            db = save_vector_store(text_chunks)
            if db is None:
                raise CustomException("Failed to create vector store")

        # Load Gemma 3
        tokenizer, llm_model = load_llm()
        if not tokenizer or not llm_model:
            raise CustomException("LLM not loaded")

        llm = GemmaLLMWrapper(tokenizer, llm_model)

        # Ensure retriever implements BaseRetriever
        retriever: BaseRetriever = db.as_retriever(search_kwargs={"k": 1})

        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            return_source_documents=False,
            chain_type_kwargs={"prompt": set_custom_prompt()},
        )

        logger.info("✅ QA chain created successfully")
        return qa_chain

    except Exception as e:
        error_message = CustomException("Failed to create QA chain", e)
        logger.error(str(error_message))
        return None


In [2]:
import torch
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForCausalLM

# Local path to Gemma 3 model
LOCAL_GEMMA_PATH = Path.home() / "Downloads" / "medical_chatbot" / "gemma_models" / "gemma-3-1b-it"

def load_llm(local_path: Path = LOCAL_GEMMA_PATH):
    """
    Load Gemma 3 LLM from a local path.
    Returns: tokenizer, model
    """
    try:
        if not local_path.exists():
            raise FileNotFoundError(f"Local model path does not exist: {local_path}")

        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(
            local_path,
            use_fast=False,
            trust_remote_code=True,
            repo_type="model",
            local_files_only=True
        )

        # Load model
        model = AutoModelForCausalLM.from_pretrained(
            local_path,
            device_map="auto" if torch.cuda.is_available() else None,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            trust_remote_code=True,
            repo_type="model",
            local_files_only=True
        ).eval()

        return tokenizer, model

    except Exception as e:
        print(f"❌ Failed to load LLM: {e}")
        return None, None


/Users/nuzkhan/Downloads/medical chatbot/medchatbot/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/nuzkhan/Downloads/medical chatbot/medchatbot/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# -----------------------------
# Quick test in Jupyter
# -----------------------------
if __name__ == "__main__":
    tokenizer, model = load_llm()
    if tokenizer and model:
        print("✅ Gemma 3 LLM loaded successfully!")

        # Quick generation test
        input_text = "Explain what diabetes is in 2 sentences."
        inputs = tokenizer(input_text, return_tensors="pt")
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=100)
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print("Input:", input_text)
        print("Output:", result)
    else:
        print("❌ Failed to load LLM")

`torch_dtype` is deprecated! Use `dtype` instead!


❌ Failed to load LLM: __init__() got an unexpected keyword argument 'repo_type'
❌ Failed to load LLM


In [22]:
# Install SentencePiece
#! pip install sentencepiece


Python(8986) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     |████████████████████████████████| 1.3 MB 7.7 MB/s eta 0:00:01
You should consider upgrading via the '/Users/nuzkhan/Downloads/medical chatbot/medchatbot/bin/python3 -m pip install --upgrade pip' command.


In [4]:
import torch
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.schema import BaseRetriever

from app.components.llm import load_llm
from app.components.vector_store import load_vector_store, save_vector_store
from app.components.pdf_loader import load_pdf_files, create_text_chunks
from app.common.logger import get_logger
from app.common.custom_exception import CustomException

logger = get_logger(__name__)

CUSTOM_PROMPT_TEMPLATE = """ Answer the following medical question in 2-3 lines maximum using only the information provided in the context.

Context:
{context}

Question:
{question}

Answer:
"""

def set_custom_prompt():
    return PromptTemplate(
        template=CUSTOM_PROMPT_TEMPLATE,
        input_variables=["context", "question"]
    )

class GemmaLLMWrapper:
    """Wrap Gemma 3 for LangChain."""
    def __init__(self, tokenizer, model):
        self.tokenizer = tokenizer
        self.model = model

    def __call__(self, prompt: str, **kwargs):
        inputs = self.tokenizer(prompt, return_tensors="pt")
        with torch.no_grad():
            outputs = self.model.generate(**inputs, max_new_tokens=kwargs.get("max_new_tokens", 256))
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

def create_qa_chain():
    try:
        logger.info("Loading vector store for context...")
        db = load_vector_store()

        # Auto-create vectorstore if missing
        if db is None:
            logger.warning("Vector store not found. Creating a new one...")
            documents = load_pdf_files()
            if not documents:
                raise CustomException("No documents found to create vector store")
            text_chunks = create_text_chunks(documents)
            db = save_vector_store(text_chunks)
            if db is None:
                raise CustomException("Failed to create vector store")

        # Load Gemma 3
        tokenizer, llm_model = load_llm()
        if not tokenizer or not llm_model:
            raise CustomException("LLM not loaded")

        llm = GemmaLLMWrapper(tokenizer, llm_model)

        # Ensure retriever implements BaseRetriever
        retriever: BaseRetriever = db.as_retriever(search_kwargs={"k": 1})

        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            return_source_documents=False,
            chain_type_kwargs={"prompt": set_custom_prompt()},
        )

        logger.info("✅ QA chain created successfully")
        return qa_chain

    except Exception as e:
        error_message = CustomException("Failed to create QA chain", e)
        logger.error(str(error_message))
        #return None


✅ HF_TOKEN loaded successfully.


In [7]:
from app.components.vector_store import load_vector_store

db = load_vector_store()
if db:
    print("✅ Vector store loaded successfully")
else:
    print("❌ Vector store missing")


'(ProtocolError('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')), '(Request ID: 1bc2536b-a168-4e6b-b71c-2b1f8a21a88a)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


❌ Vector store missing


In [8]:
from app.components.retriever import create_qa_chain
qa_chain = create_qa_chain()
if qa_chain:
    print("✅ QA chain is ready")
else:
    print("❌ QA chain failed")


❌ QA chain failed


In [9]:
from app.components.retriever import create_qa_chain

if __name__ == "__main__":
    print("🔄 Creating QA Chain...")
    qa_chain = create_qa_chain()
    if qa_chain:
        print("✅ QA Chain created successfully!")
        query = "What are the symptoms of diabetes?"
        print("🤖 Testing response:")
        response = qa_chain.invoke({"query": query})
        print("🧠 LLM Response:\n", response.get("result", "No result"))
    else:
        print("❌ QA Chain creation failed.")


🔄 Creating QA Chain...
❌ QA Chain creation failed.


In [10]:
from app.components.llm import load_llm
import torch

tokenizer, model = load_llm()
if not tokenizer or not model:
    print("❌ Failed to load Gemma LLM")
    exit()

prompt = "What are the symptoms of diabetes?"

inputs = tokenizer(prompt, return_tensors="pt")
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=256)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("LLM Response:")
print(text)


`torch_dtype` is deprecated! Use `dtype` instead!


LLM Response:
What are the symptoms of diabetes?

Diabetes symptoms can vary widely from person to person. Some people may have no symptoms at all, while others may experience significant symptoms. Here's a breakdown of common symptoms:

*   **Frequent urination:** This is often the first sign of diabetes, as the body tries to get rid of excess sugar.
*   **Excessive thirst:** High blood sugar levels can draw fluid from the body, leading to thirst.
*   **Unexplained weight loss:**  The body can't use sugar for energy, leading to weight loss despite eating normally.
*   **Increased hunger:**  The body isn't getting the energy it needs.
*   **Blurry vision:** High blood sugar can affect the lens of the eye, causing blurry vision.
*   **Slow-healing sores or cuts:** High blood sugar can impair blood flow and wound healing.
*   **Fatigue:** Feeling tired and weak is a common symptom.
*   **Nausea or vomiting:**  This can occur due to the body's response to high blood sugar.
*   **Frequent 

In [11]:
from app.components.vector_store import load_vector_store

db = load_vector_store()
if db:
    print("✅ Vector store loaded")
    retriever = db.as_retriever(search_kwargs={"k": 1})
    docs = retriever.get_relevant_documents("diabetes symptoms")
    print(f"Found {len(docs)} documents")
else:
    print("❌ Vector store missing")


❌ Vector store missing


In [16]:
from langchain.chains import RetrievalQA
from langchain.llms.base import LLM

class DummyLLM(LLM):
    @property
    def _llm_type(self):
        return "dummy"

    def _call(self, prompt: str, stop=None):
        return "Dummy answer for testing"

from app.components.vector_store import load_vector_store

db = load_vector_store()
retriever = db.as_retriever(search_kwargs={"k": 1})

qa_chain = RetrievalQA.from_chain_type(
    llm=DummyLLM(),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=False,
)

res = qa_chain({"query": "What are the symptoms of diabetes?"})
print(res)


/var/folders/fq/8zxmzsmd2_3fkx4tw93_z4v00000gn/T/ipykernel_9085/947458774.py:24: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res = qa_chain({"query": "What are the symptoms of diabetes?"})
Python(44716) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{'query': 'What are the symptoms of diabetes?', 'result': 'Dummy answer for testing'}


In [13]:
from app.components.pdf_loader import load_pdf_files, create_text_chunks
from app.components.vector_store import ensure_vector_store

documents = load_pdf_files()
chunks = create_text_chunks(documents)
db = ensure_vector_store(chunks)
print("Vector store loaded/created:", db)


Vector store loaded/created: None


In [15]:
import os
from app.config.config import DB_FAISS_PATH

print("Absolute path:", os.path.abspath(DB_FAISS_PATH))
print("Exists?", os.path.exists(DB_FAISS_PATH))


Absolute path: /Users/nuzkhan/Downloads/medical chatbot/app/components/vectorstore/db_faiss
Exists? True


In [17]:
import os
from app.config.config import DB_FAISS_PATH

print("Absolute path:", os.path.abspath(DB_FAISS_PATH))
print("Exists?", os.path.exists(DB_FAISS_PATH))


Absolute path: /Users/nuzkhan/Downloads/medical chatbot/app/components/vectorstore/db_faiss
Exists? True


from app.components.pdf_loader import load_pdf_files, create_text_chunks

docs = load_pdf_files()
print("Documents loaded:", len(docs))
chunks = create_text_chunks(docs)
print("Text chunks created:", len(chunks))


In [18]:
from app.components.pdf_loader import load_pdf_files, create_text_chunks

docs = load_pdf_files()
print("Documents loaded:", len(docs))
chunks = create_text_chunks(docs)
print("Text chunks created:", len(chunks))


Documents loaded: 0
Text chunks created: 0


In [19]:
from app.components.pdf_loader import load_pdf_files

docs = load_pdf_files()
print("Documents loaded:", len(docs))
for d in docs:
    print(d.metadata.get("source"), d.page_content[:100])


Documents loaded: 0


In [21]:
from app.components.vector_store import load_vector_store

db = load_vector_store()
if db:
    print("✅ Vector store loaded successfully")
    print("Number of documents:", len(db.index_to_docstore_id))
else:
    print("❌ Vector store missing")



✅ Vector store loaded successfully
Number of documents: 7080


In [22]:
from app.components.retriever import create_qa_chain

qa_chain = create_qa_chain()
if qa_chain:
    print("✅ QA chain created")
    response = qa_chain({"query": "What are the symptoms of diabetes?"})
    print("LLM Response:", response["result"])
else:
    print("❌ QA chain creation failed")


❌ QA chain creation failed


In [23]:
from app.components.llm import load_llm

tokenizer, model = load_llm()
if tokenizer and model:
    print("✅ Gemma loaded successfully")
else:
    print("❌ Failed to load Gemma")


✅ Gemma loaded successfully


In [27]:
from app.components.llm import load_llm

tokenizer, model = load_llm()
prompt = "Hello, how are you?"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))



Hello, how are you?

I'm a little confused about the concept of "reverse engineering" in a computer programming context. I've heard it often used in the news, and it seems to involve looking at a software program to understand how it works internally, rather


In [28]:
# performance_evaluation.py

import time
import csv
from app.components.retriever import create_qa_chain

# -----------------------------
# Configuration
# -----------------------------
TEST_QUERIES = [
    "What are the symptoms of diabetes?",
    "What is hypertension?",
    "What are the causes of asthma?",
    "How is tuberculosis treated?",
    "What are common side effects of insulin?",
]

NUM_REPEAT = 10  # number of times to repeat each query for throughput measurement
CSV_FILE = "performance_results.csv"

# -----------------------------
# Initialize QA chain
# -----------------------------
print("Initializing QA chain...")
qa_chain = create_qa_chain()
if qa_chain is None:
    raise RuntimeError("QA chain could not be created")

print("✅ QA chain ready\n")

# -----------------------------
# Measure latency for each query
# -----------------------------
latency_results = []

for query in TEST_QUERIES:
    # Warm-up run
    _ = qa_chain.invoke({"query": query})["result"]
    
    start_time = time.time()
    result = qa_chain.invoke({"query": query})["result"]
    end_time = time.time()
    
    latency = end_time - start_time
    latency_results.append({"query": query, "latency_sec": latency})
    print(f"Query: {query}")
    print(f"Result: {result}")
    print(f"Latency: {latency:.3f} sec\n")

# Average latency
avg_latency = sum(r["latency_sec"] for r in latency_results) / len(latency_results)
print(f"Average Latency: {avg_latency:.3f} sec\n")

# -----------------------------
# Measure throughput
# -----------------------------
num_queries = len(TEST_QUERIES) * NUM_REPEAT
start_time = time.time()

for _ in range(NUM_REPEAT):
    for query in TEST_QUERIES:
        _ = qa_chain.invoke({"query": query})["result"]

end_time = time.time()
total_time = end_time - start_time
throughput_per_sec = num_queries / total_time
throughput_per_min = throughput_per_sec * 60

print(f"Total Queries: {num_queries}")
print(f"Total Time: {total_time:.2f} sec")
print(f"Throughput: {throughput_per_sec:.2f} req/sec, {throughput_per_min:.2f} req/min\n")

# -----------------------------
# Save results to CSV
# -----------------------------
with open(CSV_FILE, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["query", "latency_sec"])
    writer.writeheader()
    writer.writerows(latency_results)
    writer.writerow({"query": "Average", "latency_sec": avg_latency})

print(f"✅ Latency results saved to {CSV_FILE}")
print("Performance evaluation completed.")


Initializing QA chain...


RuntimeError: QA chain could not be created